# California 2Monthly Prediction

## Imports and Functions

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.gaussian_process import GaussianProcessRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense, Dropout, GRU, Conv1D, Flatten, Reshape



from sklearn.svm import SVR
import os
import numpy as np
from xgboost import XGBRegressor
import sys

In [2]:
# Add the folder to the Python path

os.chdir("../")
# change working directory to project's root path
print(os.getcwd())

FIRST_YEAR= 1972
FREQUENCY= "2monthly"

c:\Users\marti\Desktop\data\hw_extra


In [3]:
folder_path = os.path.abspath("functions/") #INPUT_PATH)#'path_to_your_folder')  # Replace with the actual folder path
sys.path.insert(0, folder_path)

from Predictions import (
    PredictionExperiment,
    PredictionModel
)

c:\Users\marti\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [4]:
region="california"
metadata = pd.read_csv(f"data/new_features/{region}/metadata.csv")
metadata = metadata[metadata["frequency"]==FREQUENCY]
metadata

,id,filename,boxes,top_n,var_thresh,modes,frequency,season,methods,extra_indices,only_sea
60,ec28ad68,predictor_ec28ad68_1.parquet,0,5,NaN,1,2monthly,1,ps,NaN,True
61,ec28ad68,predictor_ec28ad68_2.parquet,0,5,NaN,1,2monthly,2,ps,NaN,True
62,ec28ad68,predictor_ec28ad68_3.parquet,0,5,NaN,1,2monthly,3,ps,NaN,True
63,ec28ad68,predictor_ec28ad68_4.parquet,0,5,NaN,1,2monthly,4,ps,NaN,True
64,ec28ad68,predictor_ec28ad68_5.parquet,0,5,NaN,1,2monthly,5,ps,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...
1495,31df530c,predictor_31df530c_8.parquet,0,30,0.15,3,2monthly,8,ps,PDO-ONI-SAM,True
1496,31df530c,predictor_31df530c_9.parquet,0,30,0.15,3,2monthly,9,ps,PDO-ONI-SAM,True
1497,31df530c,predictor_31df530c_10.parquet,0,30,0.15,3,2monthly,10,ps,PDO-ONI-SAM,True
1498,31df530c,predictor_31df530c_11.parquet,0,30,0.15,3,2monthly,11,ps,PDO-ONI-SAM,True


In [5]:
# kernel = RBF(length_scale=1.0) + WhiteKernel(noise_level=1)
# regressors = [LinearRegression(),RandomForestRegressor(random_state=42, n_estimators=5), RandomForestRegressor(random_state=42, n_estimators=10), SVR(kernel='rbf'),
#                 XGBRegressor(random_state=42, n_estimators=10, learning_rate=0.1),XGBRegressor(random_state=42, n_estimators=15, learning_rate=0.1), GaussianProcessRegressor(kernel=kernel, random_state=42, n_restarts_optimizer=10)]
# name_regressors = ["Linear", "RF5", "RF10", "SVR-rbf", "XGB10", "XGB15", "GPR"]
# indices_of_interest = ["HWN", "HWF", "HWD", "HWM", "HWA"]
# id_experiments = metadata["id"].unique()
# for id in id_experiments:
#     data = {i: pd.read_parquet(f"data/new_features/{region}/predictor_{id}_{i}.parquet") for i in range(1,13)}
#     experiment_1 = PredictionExperiment(data, indices_of_interest, regressors, name_regressors, 5, id)
#     experiment_1.execute_experiment()
#     experiment_1.get_metrics("r2", "prediction", show=False)
#     experiment_1.get_metrics("mape", "prediction", show=False)
#     experiment_1.get_metrics("r2", stage="training", show=False)
#     experiment_1.get_metrics("mape", stage="training", show=False)
#     experiment_1.get_metrics("r2", stage="CV", show=False)
#     experiment_1.get_metrics("mape", stage="CV", show=False)
#     #experiment_1.top_results("r2", 5, stage="prediction", top_data_path=f"data/results/{FREQUENCY}/top_results.csv")
#     #experiment_1.top_results("cv_r2", 5, stage="CV", top_data_path=f"data/results/{FREQUENCY}/top_results.csv")
#     experiment_1.save_results(f"data/{region}_results/results_sst.csv")

In [6]:
k=0
id_experiments = metadata["id"].unique()
indices_of_interest = ["HWN", "HWF", "HWD", "HWM", "HWA"]
print(len(id_experiments))
for id in id_experiments:
    k+=1
    print("Executing",id, "iter", k)
    data = {i: pd.read_parquet(f"data/new_features/{region}/predictor_{id}_{i}.parquet") for i in range(1,13)}
    # rnn_model = Sequential([
    # SimpleRNN(16, activation="tanh", input_shape=(1, len(data[1].columns) - len(indices_of_interest))),
    # Dropout(0.1),  # Regularization
    # Dense(8, activation="relu"),
    # Dense(len(indices_of_interest))  # Predict 5 indices
    # ])
    # lstm_model = Sequential([
    # LSTM(16, activation="tanh", input_shape=(1, len(data[1].columns) - len(indices_of_interest))),
    # Dropout(0.1),  # Regularization
    # Dense(8, activation="relu"),
    # Dense(len(indices_of_interest))  # Predict 5 indices
    # ])
    # Stacked LSTM
    stacked_lstm = Sequential([
        LSTM(16, activation="tanh", return_sequences=True, input_shape=(1, len(data[12].columns) - len(indices_of_interest))),
        LSTM(8, activation="tanh"),
        Dropout(0.1),
        Dense(len(indices_of_interest))
    ])
    cnn_rnn_model = Sequential([
        Conv1D(16, kernel_size=1, activation="relu", input_shape=(1, len(data[12].columns) - len(indices_of_interest))),
        Reshape((1, 16)),  # Back to time dimension
        SimpleRNN(8, activation="tanh"),
        Dropout(0.1),
        Dense(len(indices_of_interest))
    ])
    experiment_1 = PredictionExperiment(data, indices_of_interest, [stacked_lstm, cnn_rnn_model], ["SLSTM16", "CNNRNN16"], 5, id)
    experiment_1.execute_experiment()
    experiment_1.get_metrics("r2", "prediction", show=False)
    experiment_1.get_metrics("mape", "prediction", show=False)
    experiment_1.get_metrics("r2", stage="training", show=False)
    experiment_1.get_metrics("mape", stage="training", show=False)
    experiment_1.get_metrics("r2", stage="TSCV", show=False)
    experiment_1.get_metrics("mape", stage="TSCV", show=False)
    #experiment_1.top_results("r2", 5, stage="prediction", top_data_path=f"data/results/{FREQUENCY}/top_results.csv")
    #experiment_1.top_results("cv_r2", 5, stage="CV", top_data_path=f"data/results/{FREQUENCY}/top_results.csv")
    experiment_1.save_results(f"data/{region}_results/results_sst.csv")

120
Executing ec28ad68 iter 1
1/1 [==============================] - 0s 17ms/step
Executing 7617f0c6 iter 2
1/1 [==============================] - 0s 17ms/step
Executing 4536cfb3 iter 3
1/1 [==============================] - 0s 16ms/step
Executing bff90243 iter 4
1/1 [==============================] - 0s 23ms/step
Executing d0ca7d5c iter 5
1/1 [==============================] - 0s 16ms/step
Executing 80ff4834 iter 6
1/1 [==============================] - 0s 16ms/step
Executing 25f6f1b9 iter 7
1/1 [==============================] - 0s 16ms/step
Executing 9766e45c iter 8
1/1 [==============================] - 0s 14ms/step
Executing 676c8290 iter 9
1/1 [==============================] - 0s 16ms/step
Executing 6c22360e iter 10
1/1 [==============================] - 0s 16ms/step
Executing 793afce6 iter 11
1/1 [==============================] - 0s 16ms/step
Executing 64b53b65 iter 12
1/1 [==============================] - 0s 17ms/step
Executing 7ac1eb39 iter 13
1/1 [=========================

# Chile 2Monthly Prediction

In [7]:
region="chile"

In [8]:
metadata = pd.read_csv(f"data/new_features/{region}/metadata.csv")
metadata = metadata[metadata["frequency"]==FREQUENCY]
metadata.reset_index(drop=True,inplace=True)
metadata

,id,filename,boxes,top_n,var_thresh,modes,frequency,season,methods,extra_indices,only_sea
0,515c5f16,predictor_515c5f16_1.parquet,0,5,NaN,1,2monthly,1,ps,NaN,True
1,515c5f16,predictor_515c5f16_2.parquet,0,5,NaN,1,2monthly,2,ps,NaN,True
2,515c5f16,predictor_515c5f16_3.parquet,0,5,NaN,1,2monthly,3,ps,NaN,True
3,515c5f16,predictor_515c5f16_4.parquet,0,5,NaN,1,2monthly,4,ps,NaN,True
4,515c5f16,predictor_515c5f16_5.parquet,0,5,NaN,1,2monthly,5,ps,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...
1435,cf9ff510,predictor_cf9ff510_8.parquet,0,30,0.15,3,2monthly,8,ps,PDO-ONI-SAM,True
1436,cf9ff510,predictor_cf9ff510_9.parquet,0,30,0.15,3,2monthly,9,ps,PDO-ONI-SAM,True
1437,cf9ff510,predictor_cf9ff510_10.parquet,0,30,0.15,3,2monthly,10,ps,PDO-ONI-SAM,True
1438,cf9ff510,predictor_cf9ff510_11.parquet,0,30,0.15,3,2monthly,11,ps,PDO-ONI-SAM,True


In [9]:
# kernel = RBF(length_scale=1.0) + WhiteKernel(noise_level=1)
# regressors = [LinearRegression(),RandomForestRegressor(random_state=42, n_estimators=5), RandomForestRegressor(random_state=42, n_estimators=10), SVR(kernel='rbf'),
#                 XGBRegressor(random_state=42, n_estimators=10, learning_rate=0.1),XGBRegressor(random_state=42, n_estimators=15, learning_rate=0.1), GaussianProcessRegressor(kernel=kernel, random_state=42, n_restarts_optimizer=10)]
# name_regressors = ["Linear", "RF5", "RF10", "SVR-rbf", "XGB10", "XGB15", "GPR"]
# indices_of_interest = ["HWN", "HWF", "HWD", "HWM", "HWA"]
# id_experiments = metadata["id"].unique()
# for id in id_experiments:
#     data = {i: pd.read_parquet(f"data/new_features/{region}/predictor_{id}_{i}.parquet") for i in range(1,13)}
#     experiment_1 = PredictionExperiment(data, indices_of_interest, regressors, name_regressors, 5, id)
#     experiment_1.execute_experiment()
#     experiment_1.get_metrics("r2", "prediction", show=False)
#     experiment_1.get_metrics("mape", "prediction", show=False)
#     experiment_1.get_metrics("r2", stage="training", show=False)
#     experiment_1.get_metrics("mape", stage="training", show=False)
#     experiment_1.get_metrics("r2", stage="CV", show=False)
#     experiment_1.get_metrics("mape", stage="CV", show=False)
#     #experiment_1.top_results("r2", 5, stage="prediction", top_data_path=f"data/results/{FREQUENCY}/top_results.csv")
#     #experiment_1.top_results("cv_r2", 5, stage="CV", top_data_path=f"data/results/{FREQUENCY}/top_results.csv")
#     experiment_1.save_results(f"data/{region}_results/results_sst.csv")

In [10]:
k=0
id_experiments = metadata["id"].unique()
indices_of_interest = ["HWN", "HWF", "HWD", "HWM", "HWA"]
print(len(id_experiments))
for id in id_experiments:
    k+=1
    print("Executing",id, "iter", k)
    data = {i: pd.read_parquet(f"data/new_features/{region}/predictor_{id}_{i}.parquet") for i in range(1,13)}
    # rnn_model = Sequential([
    # SimpleRNN(16, activation="tanh", input_shape=(1, len(data[1].columns) - len(indices_of_interest))),
    # Dropout(0.1),  # Regularization
    # Dense(8, activation="relu"),
    # Dense(len(indices_of_interest))  # Predict 5 indices
    # ])
    # lstm_model = Sequential([
    # LSTM(16, activation="tanh", input_shape=(1, len(data[1].columns) - len(indices_of_interest))),
    # Dropout(0.1),  # Regularization
    # Dense(8, activation="relu"),
    # Dense(len(indices_of_interest))  # Predict 5 indices
    # ])
    # Stacked LSTM
    stacked_lstm = Sequential([
        LSTM(16, activation="tanh", return_sequences=True, input_shape=(1, len(data[12].columns) - len(indices_of_interest))),
        LSTM(8, activation="tanh"),
        Dropout(0.1),
        Dense(len(indices_of_interest))
    ])
    cnn_rnn_model = Sequential([
        Conv1D(16, kernel_size=1, activation="relu", input_shape=(1, len(data[12].columns) - len(indices_of_interest))),
        Reshape((1, 16)),  # Back to time dimension
        SimpleRNN(8, activation="tanh"),
        Dropout(0.1),
        Dense(len(indices_of_interest))
    ])
    experiment_1 = PredictionExperiment(data, indices_of_interest, [stacked_lstm, cnn_rnn_model], ["SLSTM16", "CNNRNN16"], 5, id)
    experiment_1.execute_experiment()
    experiment_1.get_metrics("r2", "prediction", show=False)
    experiment_1.get_metrics("mape", "prediction", show=False)
    experiment_1.get_metrics("r2", stage="training", show=False)
    experiment_1.get_metrics("mape", stage="training", show=False)
    experiment_1.get_metrics("r2", stage="TSCV", show=False)
    experiment_1.get_metrics("mape", stage="TSCV", show=False)
    #experiment_1.top_results("r2", 5, stage="prediction", top_data_path=f"data/results/{FREQUENCY}/top_results.csv")
    #experiment_1.top_results("cv_r2", 5, stage="CV", top_data_path=f"data/results/{FREQUENCY}/top_results.csv")
    experiment_1.save_results(f"data/{region}_results/results_sst.csv")

120
Executing 515c5f16 iter 1
1/1 [==============================] - 0s 19ms/step
Executing 33b6960f iter 2
1/1 [==============================] - 0s 15ms/step
Executing 8ee5f8a8 iter 3
1/1 [==============================] - 0s 16ms/step
Executing 22baf9e8 iter 4
1/1 [==============================] - 0s 15ms/step
Executing de832691 iter 5
1/1 [==============================] - 0s 16ms/step
Executing 8f0d274d iter 6
1/1 [==============================] - 0s 18ms/step
Executing c78d9d18 iter 7
1/1 [==============================] - 0s 17ms/step
Executing 7cf808b3 iter 8
1/1 [==============================] - 0s 17ms/step
Executing 9c27f2e2 iter 9
1/1 [==============================] - 0s 17ms/step
Executing a76cf7e1 iter 10
1/1 [==============================] - 0s 16ms/step
Executing d72bf2c8 iter 11
1/1 [==============================] - 0s 17ms/step
Executing 17c74f46 iter 12
1/1 [==============================] - 0s 16ms/step
Executing 4e8ef728 iter 13
1/1 [=========================